In [1]:
%load_ext autotime

In [2]:
import sys
#sys.path.append('/Users/Pradap/Documents/Research/Python-Package/enrique/')
sys.path.append('/scratch/pradap/python-work/enrique/')

time: 52.5 ms


In [3]:
import magellan as mg
import pandas as pd

DEBUG:Cloud:Log file (/u/p/r/pradap/.picloud/cloud.log) opened


time: 3.12 s


In [4]:
#mg.init_jvm('/Library/Java/JavaVirtualMachines/jdk1.8.0_45.jdk/Contents/Home/jre/lib/server/libjvm.dylib')
mg.init_jvm()

True

time: 468 ms


In [5]:
# read walmart dataset
wal =  mg.read_csv(mg.get_install_path() + '/datasets/books/walmart.csv', 
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')

time: 1.31 s


In [6]:
bwk = mg.read_csv(mg.get_install_path() + '/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')

time: 77.3 ms


In [7]:
wal.columns

Index([u'id', u'title', u'author', u'numAuthors', u'binding', u'publisher',
       u'isbn', u'pubYear', u'pubMonth', u'pubDay', u'editionNum', u'pages',
       u'volume', u'lang', u'editionDescr', u'category', u'upc11', u'upc12',
       u'upc13', u'upc14', u'description', u'url'],
      dtype='object')

time: 1.7 ms


In [8]:
bwk.columns

Index([u'id', u'title', u'author', u'numAuthors', u'binding', u'publisher',
       u'isbn', u'pubYear', u'pubMonth', u'pubDay', u'editionNum', u'pages',
       u'volume', u'lang', u'editionDescr', u'series', u'upc'],
      dtype='object')

time: 49.1 ms


In [9]:
ob = mg.OverlapBlocker()

time: 48 ms


In [10]:
mg._verbose=False
mg._percent=5

time: 52.4 ms


In [11]:
C = ob.block_tables(wal, bwk, 'title', 'title', overlap_size=3, rem_stop_words=False)

0%                          100%
[##############################] | ETA[sec]: 0.000 

time: 3min 47s



Total time elapsed: 168.966 sec


In [17]:
len(C)

8636769

time: 51.8 ms


In [18]:
bwk.dtypes

id                int64
title            object
author           object
numAuthors        int64
binding          object
publisher        object
isbn             object
pubYear         float64
pubMonth        float64
pubDay          float64
editionNum       object
pages            object
volume           object
lang             object
editionDescr     object
series           object
upc              object
dtype: object

time: 79 ms


In [14]:
E = ob.block_candset(C, 'author', 'author', overlap_size=5, qgram=3, word_level=False)

0%                          100%
[##############################] | ETA[sec]: 0.000 

time: 8min 17s



Total time elapsed: 490.754 sec


In [20]:
E.head()

,_id,ltable.id,rtable.id
158,158,194255,9780072564624
752,752,198819,9780230108189
3340,3340,204770,9780131611337
4872,4872,217679,9780160543579
4873,4873,217679,9780160543982


time: 7.33 ms


In [21]:
D = mg.debug_blocker(wal, bwk, C, pred_list_size=20)

KeyboardInterrupt: 

time: 57.8 s
